In [1]:
import time

import json5
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
from src.myedhrec import MyEDHREc

myedhred = MyEDHREc()
# Example link edhrec page : 'https://json.edhrec.com/pages/cards/dusk-dawn.json'
similar_cards_example = myedhred.get_similar_cards('Dusk // Dawn')
[x['name'] for x in similar_cards_example]

['Austere Command',
 'Solar Tide',
 'Retribution of the Meek',
 'Slaughter the Strong',
 'Fell the Mighty',
 'Citywide Bust']

In [3]:
json_cards = myedhred.get_json('Dusk // Dawn')
json_cards

{'creature': 32,
 'instant': 9,
 'sorcery': 8,
 'artifact': 9,
 'enchantment': 7,
 'battle': 0,
 'planeswalker': 1,
 'land': 33,
 'basic': 17,
 'nonbasic': 16,
 'similar': [{'aetherhub_uri': 'https://aetherhub.com/Meta/Format/Commander/?com=Austere%20Command&updated=all-time',
   'archidekt_uri': 'https://archidekt.com/search/decks?commanderName=Austere%20Command&deckFormat=3',
   'color_identity': ['W'],
   'cmc': 6.0,
   'deckstats_uri': 'https://deckstats.net/decks/search/?search_cards[]=Austere%20Command&search_format=10',
   'image_uris': [{'normal': 'https://cards.scryfall.io/normal/front/a/3/a31ffc9e-d21b-4a8f-ac67-695e38e09e3b.jpg?1706240553',
     'art_crop': 'https://cards.scryfall.io/art_crop/front/a/3/a31ffc9e-d21b-4a8f-ac67-695e38e09e3b.jpg?1706240553'}],
   'layout': 'normal',
   'moxfield_uri': 'https://moxfield.com/decks/public?format=commander&filter=Austere%20Command',
   'mtggoldfish_uri': 'https://www.mtggoldfish.com/deck/custom?mformat=commander&commander=Austere+C

## Get all cards

In [4]:
df_cards = pd.read_csv("./cards_to_study.csv",usecols=["name","setCode","uuid","name_set"])
df_cards

,name,setCode,uuid,name_set
0,Camouflage,2ED,5a1075bf-653a-595a-aab0-b1ca31eaaef6,Unlimited Edition
1,Chaos Orb,2ED,22768213-dec8-5450-89b2-865bd07ccd52,Unlimited Edition
2,False Orders,2ED,b5fc3df9-280e-5b02-80c9-d170fb55d7c7,Unlimited Edition
3,Natural Selection,2ED,592039eb-d25e-5ed4-bdd7-9b83c399e4a1,Unlimited Edition
4,Raging River,2ED,e8dd7caa-24ac-5e83-bb28-7175a9c1c9ff,Unlimited Edition
...,...,...,...,...
28311,Forest,DFT,5fdc9723-d72b-577b-8f56-4094b502e69b,Aetherdrift
28312,Island,DFT,b73151b0-e042-59df-9f0e-253c2536e672,Aetherdrift
28313,Mountain,DFT,1772f4e7-d5fa-5960-89d2-d6592aa212d5,Aetherdrift
28314,Plains,DFT,07991926-f94c-51c5-9223-4e2aabfd043b,Aetherdrift


In [5]:
df_cards["setCode"].unique()

array(['2ED', 'ARN', 'ATQ', 'LEG', 'SUM', 'DRK', 'FEM', '4BB', 'ICE',
       'BCHR', 'REN', 'RIN', 'HML', 'ALL', 'MIR', 'VIS', '5ED', 'POR',
       'WTH', 'TMP', 'STH', 'EXO', 'P02', 'USG', 'ATH', 'ULG', '6ED',
       'PTK', 'UDS', 'S99', 'MMQ', 'BRB', 'NEM', 'S00', 'PCY', 'BTD',
       'INV', 'PLS', '7ED', 'APC', 'ODY', 'DKM', 'TOR', 'JUD', 'PRM',
       'JP1', 'ONS', 'LGN', 'PLGN', 'SCG', '8ED', 'MRD', 'P04', 'DST',
       '5DN', 'PMRD', 'CHK', 'BOK', 'SOK', 'PSOK', '9ED', 'PSAL', 'RAV',
       'PHUK', 'F06', 'GPT', 'DIS', 'CSP', 'CST', 'TSP', 'TSB', 'F07',
       'P07', 'PLC', 'FUT', '10E', 'ME1', 'LRW', 'MOR', 'SHM', 'EVE',
       'ME2', 'ALA', 'CON', 'ARB', 'PARB', 'M10', 'HOP', 'ME3', 'ZEN',
       'H09', 'F10', 'G10', 'WWK', 'PWWK', 'DDE', 'ROE', 'ARC', 'M11',
       'DDF', 'SOM', 'TD0', 'PD2', 'PS11', 'F11', 'ME4', 'MBS', 'DDG',
       'NPH', 'TD2', 'CMD', 'M12', 'V11', 'DDH', 'ISD', 'PD3', 'PIDW',
       'J12', 'DKA', 'DDI', 'AVR', 'M13', 'V12', 'DDJ', 'RTR', 'GTC',
       'DD

In [16]:
from tqdm.notebook import tqdm
from src.myedhrec import format_card_name
import time
from pathlib import Path
import json
list_error = []
def load_card_and_save(card_name: str, path: Path):
	try:
		json_card : dict = myedhred.get_json(card_name)
	except Exception as e:
		if " // " in card_name:
			# Remove all part after " // "
			card_name = card_name.split(" // ")[0]
		try:
			json_card : dict = myedhred.get_json(card_name)
		except:
			return None
	return json_card

# Double for loop with tqdm, one for the set and the other for the card

pbar_set = tqdm(total=len(df_cards["setCode"].unique()))
pbar_cards = tqdm()
for set_code in df_cards["setCode"].unique():
	path_set = Path(f"./json_files/{set_code}")
	path_set.mkdir(exist_ok=True)
	pbar_set.set_description(f"Set {set_code}")
	df_cards_in_set = df_cards.query(f"setCode == '{set_code}'")
	pbar_cards.reset(total=len(df_cards_in_set))
	for card in df_cards_in_set["name"].values.tolist():
		non_formatted_path_card = path_set / f"{card}.json"
		formatted_path_card = path_set / f"{format_card_name(card)}.json"
		if not formatted_path_card.exists():
			if non_formatted_path_card.exists():
				non_formatted_path_card.rename(formatted_path_card)
				non_formatted_path_card.unlink()
			else:
				json_card = load_card_and_save(card, formatted_path_card)
				if json_card is None:
					list_error.append(card)
				else :
					with open(formatted_path_card, "w") as f:
						f.write(json.dumps(json_card))

		pbar_cards.set_description(f"Card {card}")
		pbar_cards.update()

	pbar_set.update()



  0%|          | 0/397 [00:00<?, ?it/s]

0it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
list_error

[]

In [9]:
from src.myedhrec import format_card_name
format_card_name("Rune of Protection: Artifacts")

'rune-of-protection:-artifacts'

In [8]:
df_error = pd.DataFrame(list_error, columns=["name"])
df_error.to_csv("error.csv", index=False)

In [7]:
from tqdm.notebook import tqdm
for i in tqdm(range(10)):
	time.sleep(1)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
from src.similaritygraph import SimilarityGraph

delimiter = ';'

simi_graph : SimilarityGraph= SimilarityGraph()
simi_graph.add_card('Tidal Barracuda', 5)

In [9]:
simi_graph.graph.nodes

NodeView(('Tidal Barracuda', 'Vedalken Orrery', 'Leyline of Anticipation', 'Leyline of Sanctity', 'Leyline of the Void', 'Leyline of Abundance', 'Leyline of Combustion', 'Leyline of Lifeforce', 'Leyline of Punishment', 'Winding Canyons', "Alchemist's Refuge", "Yeva, Nature's Herald", 'Teferi, Mage of Zhalfir', 'Quicken', 'Aluren', 'Vernal Equinox', 'Shimmer Myr', "Raff Capashen, Ship's Mage", 'Vivien, Champion of the Wilds', 'Venser, Shaper Savant', 'Vendilion Clique', 'Snapcaster Mage', 'Draining Whelk', 'Voidmage Husher', 'Stunt Double', 'Teferi, Time Raveler', 'Teferi, Hero of Dominaria', 'Teferi, Master of Time', 'Teferi, Temporal Archmage', 'Venser, the Sojourner', 'Narset, Parter of Veils', 'Narset Transcendent', 'Jace, the Mind Sculptor', "Teferi's Protection", "Teferi's Ageless Insight", 'Hypersonic Dragon', 'Niv-Mizzet, Dracogenius', 'Niv-Mizzet, the Firemind', 'Niv-Mizzet, Parun', "Sarkhan's Whelp", 'Enigma Drake', 'Crackling Drake'))

In [10]:
simi_graph.plot_graph().export_html('out.html', overwrite=True)


In [ ]:
simi_graph.write_graph('graph.txt')
!cat graph.txt | head -n 20

In [ ]:
new_graph = SimilarityGraph.load_graph('graph.txt')

list(new_graph.graph.edges)[:10]

In [ ]:
new_graph.plot_graph().export_html('out2.html', overwrite=True)